In [ ]:
import numpy as np

In [ ]:
data = []
document_frequency = []
with open('test/dictionary.txt') as txt:
    for line in txt:
        row = line.split()
        data.append(row[ : -1])
        document_frequency.append(row[-2])
document_frequency = np.asarray(document_frequency)        
document_frequency = document_frequency.astype(np.int)
document_frequency = np.insert(document_frequency , 0 , 0 , axis = 0)

In [ ]:
N = 5739
N_arr = np.ones(len(document_frequency)) * N
data = []
term_index = []
term_frequency = []
doc_tf = []
doc_idf = []
doc_tfidf = []
norm_doc_tfidf = []
for i in range(N) :
    term_index.append([])
    term_frequency.append([])
    doc_tf.append([])
    doc_idf.append([])
    doc_tfidf.append([])
    norm_doc_tfidf.append([])
    doc_tf[i] = np.zeros(len(document_frequency), dtype = int)
    with open('test/binary_vector/doc' + str(i) + '.txt') as txt :
        for line in txt:
            row = line.split()
            data.append(row[ : -1])
            term_index[i].append(row[0])
            term_frequency[i].append(row[-1])
    term_index[i].pop(0)
    term_frequency[i].pop(0)
    j = 0
    for index in term_index[i] :
        doc_tf[i][int(index)] = term_frequency[i][j]
        j = j + 1
    doc_idf[i] = np.log10(np.divide(N_arr , document_frequency))
    doc_tfidf[i] = np.multiply(doc_idf[i] , doc_tf[i])
    norm_doc_tfidf[i] = doc_tfidf[i] / np.linalg.norm(doc_tfidf[i])


In [ ]:
print(norm_doc_tfidf[0])

In [ ]:
import csv

with open('norm_tfidf.txt' , 'w') as f :
    csvwriter = csv.writer(f)
    csvwriter.writerows(norm_doc_tfidf)

In [ ]:
print("a")

In [ ]:

doc_idf = np.log10(np.divide(N_arr , document_frequency))

In [ ]:
 for i 
    i = 0
    for index in term_index :
        doc_tf[int(index)] = term_frequency[i]
        i = i + 1

In [ ]:
doc_tf = np.zeros(len(document_frequency) , dtype = int)

In [ ]:
doc_tfidf = np.multiply(doc_idf , doc_tf)

In [ ]:
norm_doc_tfidf = doc_tfidf / np.linalg.norm(doc_tfidf)

In [ ]:
d = []
d.append([])
print(type(d[0]))
d[0].append(1)
print(type(d[0]))

In [ ]:
print(d)

In [ ]:
print(term_frequency[5738])

拍謝頗雜亂
by pochun

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/irtm.csv')  
count_row = df.shape[0] 
count_col = df.shape[1]
print(count_row)
print(count_col)
print(df.columns)
print(df['object_class'].unique())

In [ ]:
print("Max rating \n" , df.loc[df['rating'].idxmax()] , "\n")
print("Min rating \n" , df.loc[df['rating'].idxmin()])

In [ ]:
print(df.plot.hist(bins = 100 , xlim = (-100 , 500)))

In [ ]:
print(df.groupby('object_class').count())

In [ ]:
l = [i for i in range(count_row)]
df['index'] = l

training validation testing 的切法還須調整

In [ ]:
train_df = df.iloc[4500 : count_row] 
val_df = df.iloc[3000 : 4000]
test_df = df.iloc[0 : 1000]
train_df = train_df.reset_index()
val_df = val_df.reset_index()
test_df = test_df.reset_index()
train_df = train_df.drop(columns = ['level_0'])
val_df = val_df.drop(columns = ['level_0'])
test_df = test_df.drop(columns = ['level_0'])

In [ ]:
class_test_df = test_df.groupby("object_class")
euclid_test_df = class_test_df.get_group("euclid")
thaumiel_test_df = class_test_df.get_group("thaumiel")
keter_test_df = class_test_df.get_group("keter")
safe_test_df = class_test_df.get_group("safe")
# 照此切法 testing 沒有 neutralized QQ
# neutralized_test_df = class_test_df.get_group("neutralized")

In [ ]:
class_train_df = train_df.groupby("object_class")
euclid_train_df = class_train_df.get_group("euclid")
thaumiel_train_df = class_train_df.get_group("thaumiel")
keter_train_df = class_train_df.get_group("keter")
safe_train_df = class_train_df.get_group("safe")
neutralized_train_df = class_train_df.get_group("neutralized")

In [ ]:
e_index_list_train = euclid_train_df['index'].tolist()
t_index_list_train = thaumiel_train_df['index'].tolist()
k_index_list_train = keter_train_df['index'].tolist()
s_index_list_train = safe_train_df['index'].tolist()
n_index_list_train = neutralized_train_df['index'].tolist()

以下是我的 PA3 經過些微調整

In [ ]:
import glob
import os
import math

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def term_extraction (txt):
    txt = str(txt) 
    punctuation = ["\n" , "``" , '$' , ', ' , '. ' , '; ' , '.' , '_' , ':' , "''" , '\"' , '\'' , '\"' , '(' , ')', '!' , '?' , '‘' , '*' , '&' , '`' , '\\']
    for mark in punctuation:
        txt = txt.replace(mark , ' ')
    txt = txt.replace(',' , '')
    txt = txt.replace("\'s " , " is ")
    txt = txt.replace("-" , " ")
    split_txt = []
    temp = ''
    for char in txt:
        if char == ' ':
            split_txt.append(temp)
            temp = ''
        else:
            temp += char
    if temp:
        split_txt.append(temp)
    token = []
    for s in split_txt:
        token.append(s.lower())
    ps_token = []
    for word in token:
        ps_token.append(ps.stem(word))
    stop_word = ["", "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    final = []
    for word in ps_token:
        appear = False
        for stop in stop_word:
            if word == stop:
                appear = True
        if not appear and word.isalpha():
            final.append(word)

    return final

In [ ]:
def to_dictionary (vocabulary_collection) :
    dictionary = pd.DataFrame(columns = ['term' , 'df' , 'cf'])
    df = 0
    cf = 0
    for d in vocabulary_collection:
        for t in d:
            if t not in dictionary.values:        
                for d2 in vocabulary_collection :
                    cf = cf + d2.count(t)
                    if t in d2:
                        df = df + 1
                dictionary.loc[-1] = [t , df , cf]  
                dictionary.index = dictionary.index + 1
            df = 0
            cf = 0
    dictionary.sort_values(by = ['term'] , inplace = True , ascending = True)
    dictionary = dictionary.reset_index(drop = True)
    dictionary.to_csv('pochun_testing/dictionary.txt' , index = None , sep = ' ')

In [ ]:
def to_class_dictionary (train_df , dictionary) :
    class_df = []
    for i in range (5) :
        if (i == 0) :
            cdocument_list = e_index_list_train
        elif (i == 1) :
            cdocument_list = t_index_list_train
        elif (i == 2) :
            cdocument_list = k_index_list_train
        elif (i == 3) :
            cdocument_list = s_index_list_train
        elif (i == 4) :
            cdocument_list = n_index_list_train
        else :
            print("wrong_class_index")

        class_collection = ''
        for docId in cdocument_list :     
            class_df_index = train_df[train_df['index'] == docId].index.tolist()
            index = class_df_index[0]
            string = train_df.iloc[index]['content']  
            class_collection = class_collection + string
        class_vocabulary = term_extraction(class_collection)
        c = pd.DataFrame(columns = ['term' , 'classf'])
        for t in dictionary['term'] :
            classf = class_vocabulary.count(t)
            c.loc[-1] = [t , classf]
            c.index = c.index + 1    
        class_df.append(c)
    for c in class_df :
        prob = []
        for index , row in c.iterrows() :        
            prob.append(math.log((row['classf'] + 1) / (c['classf'].sum() + 500)))
        c['probability'] = prob
    return class_df

In [ ]:
def to_class_vocabulary (train_df) :
    class_vocabulary = [] 
    
    for i in range(5):
        class_vocabulary.append([])

    for i in range (5) :
        if (i == 0) :
            cdocument_list = e_index_list_train
        elif (i == 1) :
            cdocument_list = t_index_list_train
        elif (i == 2) :
            cdocument_list = k_index_list_train
        elif (i == 3) :
            cdocument_list = s_index_list_train
        elif (i == 4) :
            cdocument_list = n_index_list_train
        else :
            print("wrong_class_index")
        
        for docId in cdocument_list :
#             print(docId)
            class_df_index = train_df[train_df['index'] == docId].index.tolist()
#             print(class_df_index)
            index = class_df_index[0]
            string = train_df.iloc[index]['content'] 
            class_vocabulary[i].append(term_extraction(string)) 
    return class_vocabulary

In [ ]:
def to_test_df (test_vocabulary_collection , dictionary) :
    test_df = []
    for d in test_vocabulary_collection :
        tdoc = pd.DataFrame(columns = ['term' , 'tf'])
        for t in d :
            tf = 0
            if (t in dictionary.values) and (t not in tdoc.values) :
                tf = d.count(t)
                tdoc.loc[-1] = [t , tf]
                tdoc.index = tdoc.index + 1    
        test_df.append(tdoc)
    return test_df

In [ ]:
# os.mkdir("pochun_testing")
all_list = []
for i in range(count_row) :
    all_list.append(str(i))
train_list = [i for i in range(4500 , count_row)]
val_list = [i for i in range(3000 , 4000)]
test_list = [i for i in range(0 , 1000)]

In [ ]:
collection = []
for docId in train_list : 
    class_df_index = df[df['index'] == docId].index.tolist()
    index = class_df_index[0]
    string = df.iloc[index]['content'] 
    collection.append(string)
vocabulary_collection = []
for i in range(len(collection)):
    vocabulary_collection.append(term_extraction(collection[i]))
to_dictionary(vocabulary_collection)

In [ ]:
dictionary = pd.read_csv('pochun_testing/dictionary.txt' , sep = " ")
zeros = [0] * len(dictionary)
dictionary['chi'] = zeros
class_vocabulary = to_class_vocabulary(train_df)
chi_table = []

for term_index in range(len(dictionary)) :
    chi_n_2 = pd.DataFrame()
    zeros13 = [-1] * 13
    chi_n_2['present'] = zeros13
    chi_n_2['absent'] = zeros13
    term = dictionary.iloc[term_index]['term']
    for class_index in range(len(class_vocabulary)) :
        present = 0
        for class_doc_index in range(len(class_vocabulary[class_index])) :
            
            if term in class_vocabulary[class_index][class_doc_index] :
                present = present + 1
        chi_n_2['present'][class_index] = present
        absent = 15 - present
        chi_n_2['absent'][class_index] = absent
    chi_score = 0
    for index , row in chi_n_2.iterrows() :
        expect = (15 / (13 * 15)) * (chi_n_2['present'].sum() / (13 * 15)) * (13 * 15)
        n = row['present']
        chi_score = chi_score + ((n - expect)**2 / expect)
        expect = (15 / (13 * 15)) * (chi_n_2['absent'].sum() / (13 * 15)) * (13 * 15)
        n = row['absent']
        chi_score = chi_score + ((n - expect)**2 / expect)
    dictionary['chi'][term_index] = chi_score
dictionary = dictionary.sort_values(by = ['chi'] , ascending = False)
dictionary = dictionary.sort_values(by = ['chi'] , ascending = False)
dictionary = dictionary.reset_index(drop = True)
dictionary = dictionary.head(500)

chi square == inf 應該有錯誤QQ(暫時查無原因QQ)

In [ ]:
print(dictionary)

In [ ]:
class_df = to_class_dictionary(train_df , dictionary)
for i in range(len(class_df)) :
    class_df[i] = class_df[i].reset_index(drop = True)

In [ ]:
print(class_df)

In [ ]:
test_collection = []
for docId in test_list:        
    class_df_index = test_df[test_df['index'] == docId].index.tolist()
    index = class_df_index[0]
    string = test_df.iloc[index]['content'] 
    test_collection.append(string)
test_vocabulary_collection = []
for i in range(len(test_collection)):
    test_vocabulary_collection.append(term_extraction(test_collection[i]))
test_dff = to_test_df(test_vocabulary_collection , dictionary)
for i in range(len(test_df)) :
    test_dff[i] = test_dff[i].reset_index(drop = True)

In [ ]:
answer = []
for d in test_dff :
    class_index = 0
    clist = []
    for c in class_df :
        if class_index == 0 :
            p = math.log(2368 / (2368 + 1024 + 231 + 1987 + 129))
        elif class_index == 1 :
            p = math.log(1024 / (2368 + 1024 + 231 + 1987 + 129))
        elif class_index == 2 :
            p = math.log(231 / (2368 + 1024 + 231 + 1987 + 129))
        elif class_index == 3 :
            p = math.log(1987 / (2368 + 1024 + 231 + 1987 + 129))
        elif class_index == 4 :
            p = math.log(129 / (2368 + 1024 + 231 + 1987 + 129))
        else :
            print("wrong class index")
        class_index = class_index + 1
        for index , row in d.iterrows() :
            t = row['term']
            if (t in c.values) and (t in dictionary.values) :
                class_df_index = c[c['term'] == t].index.tolist()
                index = class_df_index[0]
#                 print(index)
                prob = c.iloc[index]['probability']
#                 print(prob)
                p = p + prob * row['tf']
        clist.append(p)  
    maxi = -99999999
    maxIndex = -1
    for i in range(len(clist)):
        if clist[i] > maxi:
            maxi = clist[i]
            maxIndex = i + 1
    answer.append(maxIndex)
print(answer)
answer = [x - 1 for x in answer]
index = list(map(int , test_list))
output = pd.DataFrame(list(zip(test_list , answer)) , columns =['Id', 'Value'])
output.to_csv('pochun_testing/result.csv' , index = None , sep = ',')     

0 - 4 分別代表 'euclid' 'thaumiel' 'keter' 'safe' 'neutralized'
以上印出來的結果應該全部 -1(輸出的csv有減))

In [ ]:
count = 1
for c in class_df :
    c.to_csv('pochun_testi有檢/class' + str(count) + '.txt' , index = None , sep = ' ')
    count  = count + 1

In [ ]:
count = 1
for t in test_dff :
    t.to_csv('pochun_testing/test' + str(count) + '.txt' , index = None , sep = ' ')
    count  = count + 1

In [ ]:
dictionary.to_csv('pochun_testing/dic500' + str(count) + '.txt' , index = None , sep = ' ')

In [ ]:
test_df['object_class'] = test_df['object_class'].replace("euclid", 0)
test_df['object_class'] = test_df['object_class'].replace("thaumiel", 1)
test_df['object_class'] = test_df['object_class'].replace("keter", 2)
test_df['object_class'] = test_df['object_class'].replace("safe", 3)
test_df['object_class'] = test_df['object_class'].replace("neutralized", 4)


In [ ]:
count = 0
for i in range(1000) :
    if(test_df.at[i , 'object_class'] == answer[i]) :
        count = count + 1
print(count)

In [ ]:
test_df.to_csv('pochun_testing/test_df.csv' , index = None , sep = ',')